In [1]:
"""Progetto_gruppo3.ipynb
"""

#%% IMPORT SECTION
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import randint
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble 
from sklearn import tree

#%% DIRECTORY SETTINGS
# Change work directory
os.chdir('//srv0001/Risorse/Public/Gruppo3-Python')
cwd = os.getcwd()
print(cwd)


#%% PREPARE FUNCTION
def prepare_data(data):
   
    features = data.drop(["REF_DATE", "SEDOLCHK", "ICB_INDUSTRY_NUM", "ICB_SECTOR_NUM"], axis=1).astype("float64")
        
    return features


\\srv0001\Risorse\Public\Gruppo3-Python


In [2]:
#%% IMPORT DATA
#Import .csv
clean_data = pd.read_csv(r'Data/clean2.csv', sep=',')

for col in clean_data.columns:
        if clean_data[col].dtype==object and col!='SEDOLCHK':
            clean_data[col]=pd.to_numeric(clean_data[col],errors='coerce')
        
companynames = pd.read_csv(r'Data/companynames.csv', sep=',')
companynames = companynames.drop_duplicates(subset = "SEDOLCHK")

#print(clean_data.head(10))

referenceDate = clean_data["REF_DATE"].drop_duplicates().reset_index(drop = True)

print(referenceDate.head(10))

sectors_dict = {'1':'Oil & Gas','1000':'Basic Materials','2000':'Industrial',
'3000':'Consumer Good','4000':'Health Care','5000':'Consumer Services',
'6000':'Telecomunications','7000':'Utilities','8000':'Financials',
'9000':'Technology'}

0    20050429
1    20050531
2    20050630
3    20050729
4    20050831
5    20050930
6    20051031
7    20051130
8    20051230
9    20060131
Name: REF_DATE, dtype: int64


In [33]:
#%% get the rolling window
alldata = len(referenceDate.index)
months_to_train = 12;
Num_of_Try = 10

value = np.random.randint(months_to_train, alldata, size=Num_of_Try)
    
for i in value:
    All_dateToCheck = referenceDate.loc[value]
    All_startRefDate  = referenceDate.loc[value-months_to_train-1]
    All_endRefDate = referenceDate.loc[value-1]
    print(All_dateToCheck)
    date = [All_dateToCheck,All_startRefDate,All_endRefDate]
    del All_dateToCheck,All_startRefDate,All_endRefDate
    del date

171    20190731
102    20131031
107    20140331
33     20080131
145    20170531
13     20060531
151    20171130
169    20190531
144    20170428
80     20111230
Name: REF_DATE, dtype: int64
171    20190731
102    20131031
107    20140331
33     20080131
145    20170531
13     20060531
151    20171130
169    20190531
144    20170428
80     20111230
Name: REF_DATE, dtype: int64
171    20190731
102    20131031
107    20140331
33     20080131
145    20170531
13     20060531
151    20171130
169    20190531
144    20170428
80     20111230
Name: REF_DATE, dtype: int64
171    20190731
102    20131031
107    20140331
33     20080131
145    20170531
13     20060531
151    20171130
169    20190531
144    20170428
80     20111230
Name: REF_DATE, dtype: int64
171    20190731
102    20131031
107    20140331
33     20080131
145    20170531
13     20060531
151    20171130
169    20190531
144    20170428
80     20111230
Name: REF_DATE, dtype: int64
171    20190731
102    20131031
107    20140331
33     

In [29]:

startRefDate = All_startRefDate.values[1]
endRefDate =  All_endRefDate.values[1]
#clean_data['REF_DATE'] = pd.to_numeric(clean_data['REF_DATE'],errors='coerce')
mywindow = clean_data[(clean_data['REF_DATE']>=startRefDate) & (clean_data['REF_DATE']<=endRefDate)]
print(All_dateToCheck.values[1])

#%% K fold using GridSearchCV

features = prepare_data(mywindow)

x = features
y = mywindow['ICB_INDUSTRY_NUM']

parameters = {'max_depth':range(15,30)}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=8)
clf.fit(X=x, y=y)
tree_model = clf.best_estimator_

print (clf.best_score_, clf.best_params_) 


20080630
0.7892617499850445 {'max_depth': 27}


In [30]:
#%% FIT THE REAL MODEL

model = tree.DecisionTreeClassifier(max_depth = clf.best_params_.get('max_depth'))
model.fit(features, mywindow['ICB_INDUSTRY_NUM'])

#% GET THE Baseline
baseline = model.predict(prepare_data(mywindow))
base_truth = mywindow["ICB_INDUSTRY_NUM"].to_numpy(copy=True)

base_sedol = pd.DataFrame(mywindow["SEDOLCHK"])
base_sedol['ICB_INDUSTRY_NUM'] = mywindow["ICB_INDUSTRY_NUM"]
base_sedol['PREDICTED'] = baseline

rowtodrop = np.where(baseline == base_truth)[0]

base_sedol = base_sedol.drop(base_sedol.index[rowtodrop])

mybase = base_sedol.merge(companynames, left_on='SEDOLCHK', right_on='SEDOLCHK')

mybase["ICB_INDUSTRY_NUM"] = mybase["ICB_INDUSTRY_NUM"].astype(str)
mybase["ICB_INDUSTRY_NUM"] = mybase["ICB_INDUSTRY_NUM"].map(sectors_dict)
mybase["PREDICTED"] = mybase["PREDICTED"].astype(str)
mybase["PREDICTED"] = mybase["PREDICTED"].map(sectors_dict)

#%% GET THE OUTLIERS SEDOLCHK & names
test_last_month = clean_data[(clean_data.REF_DATE==All_dateToCheck.values[1])].copy()
predicted = model.predict(prepare_data(test_last_month))
g_truth = test_last_month["ICB_INDUSTRY_NUM"].to_numpy(copy=True)

outliers_sedol = pd.DataFrame(test_last_month["SEDOLCHK"])
outliers_sedol['ICB_INDUSTRY_NUM'] = test_last_month["ICB_INDUSTRY_NUM"]
outliers_sedol['PREDICTED'] = predicted

rowtodrop = np.where(predicted == g_truth)[0]

outliers_sedol = outliers_sedol.drop(outliers_sedol.index[rowtodrop])

outliers = outliers_sedol.merge(companynames, left_on='SEDOLCHK', right_on='SEDOLCHK')

outliers["ICB_INDUSTRY_NUM"] = outliers["ICB_INDUSTRY_NUM"].astype(str)
outliers["ICB_INDUSTRY_NUM"] = outliers["ICB_INDUSTRY_NUM"].map(sectors_dict)
outliers["PREDICTED"] = outliers["PREDICTED"].astype(str)
outliers["PREDICTED"] = outliers["PREDICTED"].map(sectors_dict)

print(outliers)
outliers[~outliers.isin(mybase)].dropna()

print(outliers)
filename = 'Tree_Outliers_'+str(All_dateToCheck.values[1])+'.xlsx'

#outliers.to_excel(excel_writer = filename)

lst = [outliers, mybase]
del outliers, mybase # dfs still in list
del lst 

    SEDOLCHK   ICB_INDUSTRY_NUM          PREDICTED  \
0    7145056          Oil & Gas      Consumer Good   
1    BDZZQC4    Basic Materials      Consumer Good   
2    3091357  Telecomunications  Consumer Services   
3    B197DR6         Industrial  Telecomunications   
4    7184725      Consumer Good         Technology   
..       ...                ...                ...   
111  5756029      Consumer Good         Industrial   
112  5533976  Telecomunications    Basic Materials   
113  BJ2KSG2    Basic Materials      Consumer Good   
114  4943402    Basic Materials         Industrial   
115  5654781         Technology         Industrial   

                            NAME  
0                        ENI SPA  
1          #N/A Invalid Security  
2                   BT GROUP PLC  
3                   VALLOUREC SA  
4         SWATCH GROUP AG/THE-BR  
..                           ...  
111  BAYERISCHE MOTOREN WERKE AG  
112              SWISSCOM AG-REG  
113              AKZO NOBEL N.V.  
1

In [24]:
print(m)

NameError: name 'm' is not defined

In [16]:
print(mybase)

   SEDOLCHK   ICB_INDUSTRY_NUM   PREDICTED                          NAME
0   B1VLVW7          Oil & Gas  Industrial                   AKASTOR ASA
1   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
2   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
3   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
4   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
5   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
6   3127489  Consumer Services  Industrial     MARKS & SPENCER GROUP PLC
7   4598589      Consumer Good  Industrial                CONTINENTAL AG
8   4598589      Consumer Good  Industrial                CONTINENTAL AG
9   4651459          Oil & Gas  Industrial                        OMV AG
10  4651459          Oil & Gas  Industrial                        OMV AG
11  4651459          Oil & Gas  Industrial                        OMV AG
12  4651459          Oil & Gas  Industrial         